In [39]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
import warnings
import os
import re
import time

if os.name == 'nt':
    print("This is a Windows environment.")
    rc('font', family='Malgun Gothic')
elif os.name == 'posix':
    print("This is a Mac or Linux environment.")
    rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

warnings.filterwarnings("ignore", category=UserWarning)

This is a Mac or Linux environment.


In [23]:
raw_bus =  pd.read_excel('./data/서울시버스노선별정류소정보(20241209).xlsx')
raw_bus.shape


(47642, 8)

In [24]:
bus = raw_bus.drop_duplicates(subset=['정류소명'])
bus.shape

(8605, 8)

In [26]:
bus.reset_index(drop=True, inplace=True)

In [27]:
bus

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표
0,100100416,6005,1,161000612,92702,인천공항T2-B1층,126.434018,37.466505
1,100100307,6630,1,115000916,16156,영인운수.차고지,126.817935,37.545335
2,100100281,5625,1,209000202,10195,비산체육공원,126.950645,37.410540
3,100100517,6030,1,102000319,3725,한남동,127.005237,37.537160
4,112900002,서대문12,1,112900261,13946,동원베네스트주차장.두산위브입구,126.944217,37.599114
...,...,...,...,...,...,...,...,...
8600,100100458,462,133,205000251,49608,은행동성당,127.168636,37.461838
8601,100100402,9709,134,229000725,63186,문산제일고교,126.762125,37.770446
8602,100100402,9709,135,229000724,63257,문산제일고정문,126.759725,37.771029
8603,100100402,9709,136,229000723,63187,야동동,126.757008,37.771446


In [265]:
driver = webdriver.Chrome()
base_url = 'https://ko.wikipedia.org/wiki/%EC%88%98%EB%8F%84%EA%B6%8C_%EC%A0%84%EC%B2%A0%EC%97%AD_%EB%AA%A9%EB%A1%9D'

driver.get(base_url)

In [269]:
station_list = []

for i in range(5, 34, 2):
    results = driver.find_elements(By.CSS_SELECTOR, f'#mw-content-text > div.mw-content-ltr.mw-parser-output > table:nth-child({i}) > tbody > tr')
    
    for result in results:
        text = result.text
        if re.search(r'[a-zA-Z]', text) == None:
            pass
        else:
            station_name = text.split()[0]
            station_list.append(station_name)
    time.sleep(2)

index = station_list.index("신촌(2)")
station_list[index] = '신촌'
station_list.remove("신촌(경)")
index = station_list.index("양평(중)")
station_list[index] = '양평'
station_list.remove("양평(5)")
index = station_list.index("운동장·송담대")
station_list[index] = '용인중앙시장'
        
print(len(station_list))

649


In [272]:
import requests
import json

def convert_name(name):
    
    path = '/v2/local/search/keyword.json'
    api_host = 'https://dapi.kakao.com'
    api_key = '275fa9c9b83625900bee1dc000fd7dcb'
    url = api_host+path
    
    header = {
        "Authorization" : f'KakaoAK {api_key}'
    }
    
    body = {
    "format" : "json",
    'query' : name,
    }

    try:
        response = requests.get(url=url, params=body, headers=header, timeout=10)  # 10초 타임아웃
        response.raise_for_status()  # HTTP 오류가 발생하면 예외 발생
    except requests.exceptions.Timeout:
        print("Request timed out")
        return {}
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return {}

    parsed_data = json.loads(response.text)

    return parsed_data

In [273]:
addr_list = []
name_list = []
x_list = []
y_list = []

for station in station_list:
    data = convert_name(station + "역")
    count = data['meta']['total_count']
    documents = data['documents']
    for doc in documents:
        if doc['category_group_code'] == 'SW8':
            address = doc['road_address_name']
            name = doc['place_name']
            x = doc['x']
            y = doc['y']
            addr_list.append(address)
            name_list.append(name)
            x_list.append(x)
            y_list.append(y)
            break
        elif '기차역' in doc['category_name']:
            address = doc['address_name']
            name = doc['place_name']
            x = doc['x']
            y = doc['y']
            addr_list.append(address)
            name_list.append(name)
            x_list.append(x)
            y_list.append(y)
            break

print(len(addr_list), len(name_list), len(x_list), len(y_list))

649 649 649 649


In [274]:
pd.DataFrame(data={
    "Name" : name_list,
    'Address' : addr_list,
    'LO' : x_list,
    'LA' : y_list
})

,Name,Address,LO,LA
0,가능역 1호선,경기 의정부시 평화로 633,127.044288264841,37.7483963196456
1,가락시장역 8호선,서울 송파구 송파대로 지하 257,127.118262745146,37.4930992522183
2,가산디지털단지역 7호선,서울 금천구 벚꽃로 309,126.882661758356,37.4803959660982
3,가양역 9호선,서울 강서구 양천로 지하 485,126.85442142615852,37.56143311719883
4,가오리역 우이신설선,서울 강북구 삼양로 426,127.01676896839822,37.6415440572233
...,...,...,...,...
644,효자역 의정부경전철,경기 의정부시 부용로 159-40,127.077226606947,37.7539905661041
645,효창공원앞역 6호선,서울 용산구 백범로 지하 287,126.96139810075,37.5393087503306
646,흑석역 9호선,서울 동작구 현충로 지하 90,126.963463063008,37.5091654719404
647,흥선역 의정부경전철,경기 의정부시 호국로 1197,127.03717589317614,37.74328307869396


In [31]:
raw_subway = pd.read_excel('./data/전체_도시철도운행정보_20240930.xlsx')
raw_subway.shape

(206268, 13)

In [33]:
raw_subway['노선명'].value_counts()

노선명
경인선                 36213
안산과천선               33081
일산선                 29065
수인분당선               25841
경부선                 24321
경의중앙선               16483
서해선                  5936
동해선                  4340
경춘선                  4189
경강선                  2545
서울 도시철도 2호선          1814
서울 도시철도 1호선          1570
에버라인                 1060
부산 도시철도 1호선          1047
부산 도시철도 2호선           998
수도권 경량도시철도 우이신설선      940
부산 도시철도 3호선           904
대구도시철도 3호선            896
서울 도시철도 4호선           896
부산 도시철도 4호선           884
서울 도시철도 9호선           880
신분당선                  869
대구도시철도 2호선            840
인천지하철 2호선             840
대구도시철도 1호선            840
김포골드라인                812
의정부                   794
서울 도시철도 3호선           776
서울 도시철도 5호선           770
서울 도시철도 7호선           752
인천국제공항선               708
수도권 경량도시철도 신림선        696
광주 도시철도 1호선           648
인천지하철 1호선             592
서울 도시철도 6호선           591
서울 도시철도 8호선           566
대전 도시철도 1호선           460
수도권 도시철도 7호선          416
부산김해경전철 

In [ ]:
raw_subway

,열차번호,노선번호,노선명,운행구간기점명,운행구간종점명,운행유형,요일구분,운행구간정거장,정거장도착시각,정가장출발시각,운행속도,운영기관전화번호,데이터기준일자
0,K7501,I41K5,경강선,신판교,여주,일반,평일,신판교,05:30:00,05:30:00,45.4,NaN,2024-08-31 00:00:00
1,K7501,I41K5,경강선,신판교,여주,일반,평일,성남,05:32:00,05:32:30,45.4,NaN,2024-08-31 00:00:00
2,K7501,I41K5,경강선,신판교,여주,일반,평일,신이매,05:34:00,05:34:30,45.4,NaN,2024-08-31 00:00:00
3,K7501,I41K5,경강선,신판교,여주,일반,평일,삼동,05:39:30,05:40:00,45.4,NaN,2024-08-31 00:00:00
4,K7501,I41K5,경강선,신판교,여주,일반,평일,경광주,05:44:00,05:45:00,45.4,NaN,2024-08-31 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
206263,Y429,LE1,에버라인,강남대,전대·에버랜드,일반,휴일,001-강남대+002-지석+003-어정+004-동백+005-초당+006-삼가+007...,001-:/002-23:22/003-23:23/004-23:25/005-23:19/...,001-23:21/002-23:23/003-23:24/004-23:26/005-23...,NaN,031-329-3500,2022-04-28 00:00:00
206264,Y430,LE1,에버라인,둔전,기흥(백남준아트센터),일반,휴일,001-둔전+002-보평+003-고진+004-운동장·송담대+005-김량장+006-명...,001-23:12/002-23:13/003-23:15/004-23:19/005-23...,001-23:13/002-23:14/003-23:16/004-23:20/005-23...,NaN,031-329-3500,2022-04-28 00:00:00
206265,Y431,LE1,에버라인,강남대,전대·에버랜드,일반,휴일,001-강남대+002-지석+003-어정+004-동백+005-초당+006-삼가+007...,001-:/002-23:33/003-23:35/004-23:37/005-23:29/...,001-23:32/002-23:34/003-23:36/004-23:38/005-23...,NaN,031-329-3500,2022-04-28 00:00:00
206266,Y432,LE1,에버라인,둔전,기흥(백남준아트센터),일반,휴일,001-둔전+002-보평+003-고진+004-운동장·송담대+005-김량장+006-명...,001-23:22/002-23:23/003-23:25/004-23:29/005-23...,001-23:23/002-23:24/003-23:26/004-23:30/005-23...,NaN,031-329-3500,2022-04-28 00:00:00
